In [241]:
import numpy as np
import pandas as pd

In [242]:
def node_entropy(x):
    ent = 0
    #labels 0 1
    for i in range(2):
        class_count=len(x[x[:,2]==int(i)])
        try:
            #get the probability of each class
            pmi = class_count/x.shape[0]
        except:
            pmi = 0
        if pmi != 0:
            #calculating entropy
            ent -= (pmi)*np.log2(pmi)
        else:
            pass
    return ent

In [243]:
def split_entropy(x, val, i):
    lmp = 0
    left_branch = x[x[:, i] <= val]
    right_branch = x[x[:, i] > val]
    lmp += (left_branch.shape[0]/x.shape[0]) * node_entropy(left_branch)
    lmp += (right_branch.shape[0]/x.shape[0]) * node_entropy(right_branch)
    return lmp

In [244]:
def split_attribute(x):
    minent = np.inf
    for i in range(2):
        for val in x[:, int(i)]:
            ent = split_entropy(x, val, i)
            if ent < minent:
                minent = ent
                best_f = i
                threshold = val
    return (best_f, threshold)

In [245]:
def generate_tree(x, theta=0):
    if node_entropy(x) <= theta:
        entropy = node_entropy(x)
        majority_class = 1 if x[x[:,2] == 1].shape[0] > x[x[:,2] == 2].shape[0] else 0
        return (majority_class, entropy, None)
    else:
        i, xi = split_attribute(x) # attribute idx and split entropy

        Tree = {}
        x_left = x[x[:, int(i)] <= xi]
        x_right = x[x[:, int(i)] > xi]

        Tree[(i, xi, True)] = generate_tree(x_left) # Left branch
        Tree[(i, xi, False)] = generate_tree(x_right) # Right branch

        return Tree

In [246]:
def classify_point(Tree, point):
    
    if type(Tree) != dict:
        return Tree
    else:
        for d in Tree:
            i, x, binary = d

        if point[int(i)] <= x:
            return classify_point(Tree[(i, x, True)], point)
        else:
            return classify_point(Tree[(i, x, False)], point)

In [247]:
def check_accuracy(Tree, X_check):
    right = 0

    total = 0
    for i in X_check:
        print('Data',i[0], i[1], i[2])
        print('Tree ',classify_point(Tree, i[:2]))
        if classify_point(Tree, i[:2])[0] == i[2]: 
            right += 1
        total +=1
    return right/total

In [248]:
X = np.array([[2,1,0], [1,2,0], [1.5,3,1], [3,3, 1], [4,4,1]])
X_test = np.array([[0.5,0.5,1], [2,6,0]])
Tree = generate_tree(X)

In [249]:
Tree

{(1, 2.0, True): (0, 0.0, None), (1, 2.0, False): (1, 0.0, None)}

In [250]:
X_test

array([[0.5, 0.5, 1. ],
       [2. , 6. , 0. ]])

In [251]:
print(check_accuracy(Tree, np.array([[3,2.1,1], [0.5,0.5,0]])))

Data 3.0 2.1 1.0
Tree  (1, 0.0, None)
Data 0.5 0.5 0.0
Tree  (0, 0.0, None)
1.0


In [252]:
Tree

{(1, 2.0, True): (0, 0.0, None), (1, 2.0, False): (1, 0.0, None)}